In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chinese-couplets/couplet/vocabs
/kaggle/input/chinese-couplets/couplet/test/out.txt
/kaggle/input/chinese-couplets/couplet/test/in.txt
/kaggle/input/chinese-couplets/couplet/test/.in.txt.swp
/kaggle/input/chinese-couplets/couplet/test/.out.txt.swp
/kaggle/input/chinese-couplets/couplet/train/out.txt
/kaggle/input/chinese-couplets/couplet/train/in.txt


2. 尝试encoder hidden state不同的返回形式（concat和add），使用add

In [2]:
# 1、处理数据
import pickle
def get_data_list(in_path, out_path):
    with open(in_path) as in_file, open(out_path) as out_file:
        enc_data, dec_data = [], []
        for line in list(zip(in_file, out_file)):
            enc_data.append(line[0].strip().split())
            dec_data.append(['<s>'] + line[1].strip().split() + ['</s>'])
        return enc_data, dec_data
# 训练数据:770491
train_enc_data, train_dec_data = get_data_list('/kaggle/input/chinese-couplets/couplet/train/in.txt', 
                                               '/kaggle/input/chinese-couplets/couplet/train/out.txt')
# 测试数据:4000
test_enc_data, test_dec_data = get_data_list('/kaggle/input/chinese-couplets/couplet/test/in.txt', 
                                               '/kaggle/input/chinese-couplets/couplet/test/out.txt')
# # 加载字典
# with open('/kaggle/input/chinese-couplets/couplet/vocabs') as f:
#     word_list = ['PAD', 'UNK'] + [word.strip() for word in f]
#     vocab = {word:i for i, word in enumerate(word_list)}

In [3]:
def get_vocab(train_data, test_data):
    word_set = set()
    for words in (train_data + test_data):
        word_set.update(words)
    word_list = ['PAD', 'UNK'] + list(word_set)
    return {word:i for i, word in enumerate(word_list)}
enc_vocab = get_vocab(train_enc_data, test_enc_data)
dec_vocab = get_vocab(train_dec_data, test_dec_data)
# 保存词典
with open("enc_dec_vocab.pkl", "wb") as f:
    pickle.dump((enc_vocab, dec_vocab), f)

In [4]:
# 2、定义模型
import torch.nn as nn
import torch

class Encoder(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, dropout):
        super().__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, emb_size)
        self.rnn = nn.GRU(emb_size, hidden_size, batch_first=True, bidirectional=True)
    
    def forward(self, enc_idxs):
        embedded = self.embedding(enc_idxs)
        # output: [batch_size, seq_len, hidden_size * 2]
        # h_n: [num_layers * 2, batch_size, hidden_size]
        outputs, h_n = self.rnn(embedded)
        outputs = outputs[:,:,:self.hidden_size] + outputs[:,:,self.hidden_size:]
        # 返回值求和: [batch_size, hidden_size]
        return outputs, h_n.sum(dim=0)

class Attention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, enc_outputs, dec_outputs):
        # enc_outputs: [batch_size, enc_seq_len, hidden_size]
        # dec_outputs: [batch_size, dec_seq_len, hidden_size]
        a_t = torch.bmm(enc_outputs, dec_outputs.permute(0, 2, 1)) # [batch_size, enc_seq_len, dec_seq_len]
        a_t = torch.softmax(a_t, dim=1) # [batch_size, enc_seq_len, dec_seq_len]
        c_t = torch.bmm(a_t.permute(0, 2, 1), enc_outputs) # [batch_size, dec_seq_len, hidden_size * 2]
        return c_t
    
class Decoder(nn.Module):
    def __init__(self, input_size, emb_size, hidden_size, dropout):
        super().__init__()
        self.embedding = nn.Embedding(input_size, emb_size)
        self.rnn = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.attention = Attention()
        self.attention_fc = nn.Linear(hidden_size * 2, hidden_size)
        self.act = nn.Tanh()
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, dec_idxs, h_0, enc_outputs):
        embedded = self.embedding(dec_idxs)
        # dec_output: [batch_size, seq_len, hidden_size]
        # h_n: [num_layers, batch_size, hidden_size]，返回最后一个时间步的隐藏状态，用于进行推理
        dec_outputs, h_n = self.rnn(embedded, h_0.unsqueeze(0))
        c_t = self.attention(enc_outputs, dec_outputs) # [batch_size, seq_len, hidden_size]
        cat_outputs = torch.cat((c_t, dec_outputs), dim=2) # [batch_size, seq_len, hidden_size * 2]
        outputs = self.attention_fc(cat_outputs) # [batch_size, seq_len, hidden_size]
        outputs = self.act(outputs) # [batch_size, seq_len, hidden_size]
        logits = self.fc(outputs) # [batch_size, seq_len, input_size]
        return logits, h_n
    
class Seq2Seq(nn.Module):
    def __init__(self, enc_input_size, dec_input_size, emb_size, hidden_size, dropout=0.3):
        super().__init__()
        self.encoder = Encoder(enc_input_size, emb_size, hidden_size, dropout)
        self.decoder = Decoder(dec_input_size, emb_size, hidden_size, dropout)

    def forward(self, enc_idxs, dec_idxs):
        enc_outputs, h_0 = self.encoder(enc_idxs)
        outputs, h_n = self.decoder(dec_idxs, h_0, enc_outputs)
        return outputs, h_n

# if __name__ == "__main__":
#     seq2seq = Seq2Seq(200, 300, 70, 128)
#     enc_idxs = torch.randint(0, 200, (3, 10))
#     dec_idxs = torch.randint(0, 300, (3, 12))
#     outputs, h_n = seq2seq(enc_idxs, dec_idxs)
#     print(outputs.shape) # [3, 12, 300]
#     print(h_n.shape) # [1, 3, 128]

In [5]:
# 3、模型训练
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
# from EncoderDecoderModel import Seq2Seq
from torch.utils.tensorboard import SummaryWriter

# 定义格式化函数
def format_batch(enc_vocab, dec_vocab):
    def format_batch_fn(batch):
        enc_ids, dec_ids, target_ids = [], [], []
        for enc_line, dec_line in batch:
            enc_input = [enc_vocab.get(token, enc_vocab['UNK']) for token in enc_line]
            dec_input = [dec_vocab.get(token, dec_vocab['UNK']) for token in dec_line]
            enc_ids.append(torch.tensor(enc_input))
            dec_ids.append(torch.tensor(dec_input[:-1]))
            target_ids.append(torch.tensor(dec_input[1:]))  # 目标是输入序列的偏移
        enc_inputs = pad_sequence(enc_ids, batch_first=True, padding_value=enc_vocab['PAD'])
        dec_inputs = pad_sequence(dec_ids, batch_first=True, padding_value=dec_vocab['PAD'])
        targets = pad_sequence(target_ids, batch_first=True, padding_value=dec_vocab['PAD'])
        return enc_inputs, dec_inputs, targets
    return format_batch_fn

2025-04-27 07:27:13.156619: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745738833.386008      18 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745738833.453274      18 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
train_dataloader = DataLoader(list(zip(train_enc_data, train_dec_data)), batch_size=256, shuffle=True, collate_fn=format_batch(enc_vocab, dec_vocab))
test_dataloader = DataLoader(list(zip(test_enc_data, test_dec_data)), batch_size=256, shuffle=False, collate_fn=format_batch(enc_vocab, dec_vocab))
writer = SummaryWriter(log_dir='/kaggle/working/runs/add')
emb_size = 100
hidden_size = 512
epochs = 10
model = Seq2Seq(len(enc_vocab), len(dec_vocab), emb_size, hidden_size)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [7]:
for epoch in range(epochs):
        # 训练模型
    model.train()
    train_bar = tqdm(train_dataloader)
    for i, (enc_inputs, dec_inputs, targets) in enumerate(train_bar):
        enc_inputs, dec_inputs, targets = enc_inputs.to(device), dec_inputs.to(device), targets.to(device)
        out, _ = model(enc_inputs, dec_inputs)
        # targets: [batch_size, seq_len]
        # out: [batch_size, seq_len, vocab_size]
        loss = loss_fn(out.view(-1, out.size(-1)), targets.view(-1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        writer.add_scalar("loss", loss.item(), epoch * len(train_dataloader) + i)
        train_bar.set_description(f"Epoch {epoch + 1}, Loss: {loss.item()}")
    # 测试模型
    model.eval()
    with torch.no_grad():
        total = 0
        correct = 0
        for enc_inputs, dec_inputs, targets in test_dataloader:
            enc_inputs, dec_inputs, targets = enc_inputs.to(device), dec_inputs.to(device), targets.to(device)
            out, _ = model(enc_inputs, dec_inputs)
            pred = torch.argmax(out, dim=-1)
            # 标记非填充位置
            non_padding_mask = (targets != 0)
            correct += (pred[non_padding_mask] == targets[non_padding_mask]).sum().item()
            total += non_padding_mask.sum().item()
        accuracy = 100 * correct / total
        print(f"Epoch {epoch + 1}, Accuracy: {accuracy}, Total: {total}, Correct: {correct}")
        writer.add_scalar("accuracy", accuracy, epoch)
writer.close()
# 保存模型
torch.save(model.state_dict(), "model_add.pth")

Epoch 1, Loss: 1.4131219387054443: 100%|██████████| 3010/3010 [05:47<00:00,  8.66it/s]


Epoch 1, Accuracy: 32.7315617176969, Total: 41544, Correct: 13598


Epoch 2, Loss: 1.6520153284072876: 100%|██████████| 3010/3010 [06:10<00:00,  8.12it/s]


Epoch 2, Accuracy: 35.16031195840554, Total: 41544, Correct: 14607


Epoch 3, Loss: 1.3355357646942139: 100%|██████████| 3010/3010 [06:13<00:00,  8.06it/s]


...

Epoch 8, Accuracy: 36.9632197188523, Total: 41544, Correct: 15356


Epoch 9, Loss: 1.0886945724487305: 100%|██████████| 3010/3010 [06:14<00:00,  8.04it/s]


Epoch 9, Accuracy: 36.7947236664741, Total: 41544, Correct: 15286


Epoch 10, Loss: 1.32445228099823: 100%|██████████| 3010/3010 [06:14<00:00,  8.04it/s]


Epoch 10, Accuracy: 36.700847294434816, Total: 41544, Correct: 15247
